# Project Template
##### To get started with the project, go to the workspace on the next page, where you'll find the project template (a Jupyter notebook file). You can work on your project and submit your work through this workspace.

The project template includes one Jupyter Notebook file, in which:

##### you will process the event_datafile_new.csv dataset to create a denormalized dataset
##### you will model the data tables keeping in mind the queries you need to run
##### you have been provided queries that you will need to model your data tables for
##### you will load the data into tables you create in Apache Cassandra and run your queries

In [1]:
# required packages 
import pandas as pd
import numpy as np
import glob
import os
import cassandra
from cassandra.cluster import Cluster
import csv

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/nikhil/Desktop/learning/Data_engineering_Udacity/Part 1 Data modeling/Project 2 NoSql


In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
#         print(files)
        files = glob.glob(os.path.join(root,'*.csv'))
#         print(files)
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [4]:
all_files=get_files(filepath)

In [5]:
print(len(all_files))

30


#### Read the csv and write to file called "event_datafile_new"

In [6]:
df=pd.read_csv(all_files[0])

In [7]:
for i in all_files[1:]:
    df=pd.concat([df,pd.read_csv(i)])

In [8]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userId
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541020e+12,583,Sehr kosmisch,200,1.542240e+12,26.0
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541020e+12,583,The Big Gundown,200,1.542240e+12,26.0
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541020e+12,583,Marry Me,200,1.542240e+12,26.0
3,NaN,Logged In,Wyatt,M,0,Scott,NaN,free,"Eureka-Arcata-Fortuna, CA",GET,Home,1.540870e+12,563,NaN,200,1.542250e+12,9.0
4,NaN,Logged In,Austin,M,0,Rosales,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.541060e+12,521,NaN,200,1.542250e+12,12.0


In [9]:
df=df.dropna(subset=['artist'])

In [10]:
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userId
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541020e+12,583,Sehr kosmisch,200,1.542240e+12,26.0
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541020e+12,583,The Big Gundown,200,1.542240e+12,26.0
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541020e+12,583,Marry Me,200,1.542240e+12,26.0
5,Sony Wonder,Logged In,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540490e+12,597,Blackbird,200,1.542250e+12,61.0
9,Van Halen,Logged In,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",PUT,NextSong,1.540790e+12,602,Best Of Both Worlds (Remastered Album Version),200,1.542260e+12,80.0
10,Magic Sam,Logged In,Tegan,F,3,Levine,132.04853,paid,"Portland-South Portland, ME",PUT,NextSong,1.540790e+12,602,Call Me If You Need Me,200,1.542260e+12,80.0
11,Edward Sharpe & The Magnetic Zeros,Logged In,Tegan,F,4,Levine,306.31138,paid,"Portland-South Portland, ME",PUT,NextSong,1.540790e+12,602,Home,200,1.542260e+12,80.0
12,Usher featuring will.i.am,Logged In,Tegan,F,5,Levine,395.72853,paid,"Portland-South Portland, ME",PUT,NextSong,1.540790e+12,602,OMG,200,1.542260e+12,80.0
14,Helen Reddy,Logged In,Tegan,F,7,Levine,176.50893,paid,"Portland-South Portland, ME",PUT,NextSong,1.540790e+12,602,Candle On The Water,200,1.542260e+12,80.0
15,Taylor Swift,Logged In,Tegan,F,8,Levine,201.06404,paid,"Portland-South Portland, ME",PUT,NextSong,1.540790e+12,602,Our Song,200,1.542260e+12,80.0


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

In [11]:
df=df[['artist', 'firstName', 'gender', 'itemInSession', 'lastName','length', 'level', 'location','sessionId', 'song', 'userId']]

In [12]:
df.to_csv('event_datafile_new.csv')

### Now for the cassandra part

In [13]:
try:
    cluster=Cluster(['127.0.0.1'])
    session=cluster.connect()
except Exception as e:
    print(e)

In [14]:
session.execute("create keyspace if not exists sparkify with replication={'class':'SimpleStrategy','replication_factor':1}")


In [15]:
session.set_keyspace('sparkify')

## Note model is based on the queries

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [16]:
df.columns

Index(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
       'level', 'location', 'sessionId', 'song', 'userId'],
      dtype='object')

# Query #1

### Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

To answer this question we will need to obtain (select) the artist name, song name, and song length from out table, and we will need to filter by sessionId and itemInSession.
In CQL our query looks like:

*SELECT artist, song_title, song_length FROM session_songs WHERE sessionId = 338 AND itemInSession = 4*

- We will name our table **session_songs**
- Our primary key will consist of partition key sessionId, and clustering key itemInSession so that we can filter by this attributes later on.
- The columns of our table will be: sessionId, itemInSession, artist, song_title and song_length.


In [17]:
session.execute("create table if not exists session_songs(sessionId int,itemInSession int,artist text,song text,length float,primary key(sessionId,itemInSession))")

In [18]:
file=pd.read_csv('event_datafile_new.csv')

In [19]:
query1=file[['sessionId','itemInSession','artist','song','length']]

In [20]:
query1.columns=['sessionId','itemInSession','artist','song','length']
query1.head()

,sessionId,itemInSession,artist,song,length
0,583,0,Harmonia,Sehr kosmisch,655.77751
1,583,1,The Prodigy,The Big Gundown,260.07465
2,583,2,Train,Marry Me,205.45261
3,597,0,Sony Wonder,Blackbird,218.06975
4,602,2,Van Halen,Best Of Both Worlds (Remastered Album Version),289.38404


In [21]:
q="INSERT INTO session_songs (sessionId, itemInSession, artist, song,length)VALUES (%s, %s, %s, %s, %s)"
for i in query1.values:
    sessionId, itemInSession, artist, song_title, song_length=int(i[0]),int(i[1]),i[2],i[3],float(i[4])
#     print(type(sessionId),type( itemInSession),type( artist), song_title, type(song_length))
    session.execute(q,(sessionId, itemInSession, artist, song_title, song_length))
    

In [22]:
rows = session.execute("""SELECT artist, song, length FROM session_songs WHERE sessionId = 338 AND itemInSession = 4""")

for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


# Query #2

### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

To answer this question we will need to obtain (select) the artist name, song name, user name and user lastname from out table, we will need to filter by userId and sessionId, and order by itemInSession. 

In CQL our query looks like:

*SELECT itemInSession, artist, song, firstName, lastName FROM user_songs WHERE userId = 10 AND sessionId = 182*

- We will name our table **user_songs**
- Our primary key will consist of composite partition key userId, sessionId. The reason for this is that if we only use userId as partition key, the sessionid which belongs to the same user will be put into different nodes, which will have the performance issue when the volume of data is large.
- Our clustering key will be itemInSession so that our results are order by it.
- The columns of our table will be: userId, sessionId, itemInSession, artist, song and firstName and lastName.


In [23]:
df.columns

Index(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
       'level', 'location', 'sessionId', 'song', 'userId'],
      dtype='object')

In [24]:
session.execute("create table if not exists user_songs(userid int,sessionId int,itemInSession int,artist text,firstName text,lastName text,primary key((userId,sessionId),itemInSession))")

In [25]:
query2=df[['userId','sessionId','itemInSession' ,'artist','firstName' ,'lastName' ]]
query2.columns=['userId','sessionId','itemInSession' ,'artist','firstName' ,'lastName' ]
query2.head()

,userId,sessionId,itemInSession,artist,firstName,lastName
0,26.0,583,0,Harmonia,Ryan,Smith
1,26.0,583,1,The Prodigy,Ryan,Smith
2,26.0,583,2,Train,Ryan,Smith
5,61.0,597,0,Sony Wonder,Samuel,Gonzalez
9,80.0,602,2,Van Halen,Tegan,Levine


In [26]:
q="INSERT INTO user_songs (userid, sessionId, itemInSession, artist,firstName,lastName)VALUES (%s, %s, %s, %s, %s,%s)"
for i in query2.values:
#     print(int(i[0]))
    userid, sessionId, itemInSession, artist,firstName,lastName=int(i[0]),int(i[1]),int(i[2]),i[3],i[4],i[5]
#     print(type(sessionId),type( itemInSession),type( artist), song_title, type(song_length))
    session.execute(q,(userid, sessionId, itemInSession, artist,firstName,lastName))
    

In [27]:
rows = session.execute("""SELECT itemInSession, artist, firstName, lastName FROM user_songs WHERE userId = 10 AND sessionId = 182""")

for row in rows:
    print(row.iteminsession, row.artist,  row.firstname, row.lastname)

0 Down To The Bone Sylvie Cruz
1 Three Drives Sylvie Cruz
2 Sebastien Tellier Sylvie Cruz
3 Lonnie Gordon Sylvie Cruz


# Query #3

### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

To answer this question we will need to obtain (select) the user first name and lastname from out table, and we will need to filter by song name. As user name and lastname, in large datasets, are not unique, we will add the column userId to uniquely identify users.

In CQL our query looks like:

*SELECT firstName, lastName FROM app_history WHERE song = 'All Hands Against His Own'*

- We will name our table **app_history**
- Our primary key will consist of partition key song, and clustering key userId. This uniquely identifies our rows.
- The columns of our table will be: song, firstName, lastName and userId.


In [28]:
session.execute("create table if not exists app_history(song text,firstName text,lastName text,userId int,primary key(song,userId))")

In [29]:
query3=df[['song','firstName','lastName','userId']]
query3.head()

,song,firstName,lastName,userId
0,Sehr kosmisch,Ryan,Smith,26.0
1,The Big Gundown,Ryan,Smith,26.0
2,Marry Me,Ryan,Smith,26.0
5,Blackbird,Samuel,Gonzalez,61.0
9,Best Of Both Worlds (Remastered Album Version),Tegan,Levine,80.0


In [30]:
q="INSERT INTO app_history (song,firstName,lastName,userId)VALUES (%s, %s, %s,%s)"
for i in query3.values:
#     print(int(i[3]))
    userid, sessionId, itemInSession,u=i[0],i[1],i[2],int(i[3])
#     print(userid, sessionId, itemInSession,u)
    session.execute(q,(userid, sessionId, itemInSession,u))
    

In [31]:
rows = session.execute("""SELECT firstName, lastName FROM app_history WHERE song = 'All Hands Against His Own'""")

for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


## Drop all the table 

In [32]:
session.execute("drop table session_songs")
session.execute("drop table user_songs")
session.execute("drop table app_history")

In [33]:
session.shutdown()
cluster.shutdown()